Bài toán phân loại ung thư vú với Perceptron (SLP)

In [ ]:
# Cell 1 — Imports & cấu hình chung
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
import torch.optim as optim

# Seed cố định để tái lập
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# Thiết lập style cho biểu đồ
plt.style.use('seaborn-v0_8')


In [ ]:
# Cell 2 — Tải dữ liệu & chuẩn hoá
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = data.feature_names
target_names = data.target_names

# Chuẩn hoá StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Tách 60/20/20: train (60%), val (20%), test (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=SEED, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=SEED, stratify=y_train_val
)

print(f"Shapes — X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")


In [ ]:
# Cell 3 — PyTorch tensors & mô hình Perceptron
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

class Perceptron(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)  # SLP: 1 lớp tuyến tính

    def forward(self, x):
        return self.linear(x)  # logits, dùng BCEWithLogitsLoss

model = Perceptron(input_dim=X.shape[1])
criterion = nn.BCEWithLogitsLoss()

# Ràng buộc lr ∈ {1e-1, 1e-2}; mặc định 1e-2 (ổn định hơn)
LR = 1e-2
optimizer = optim.SGD(model.parameters(), lr=LR)

MAX_EPOCHS = 50


In [ ]:
# Cell 4 — Huấn luyện Perceptron với SGD (epoch ≤ 50)
train_losses = []
val_losses = []

for epoch in range(1, MAX_EPOCHS + 1):
    model.train()
    optimizer.zero_grad()
    logits = model(X_train_tensor)
    loss = criterion(logits, y_train_tensor)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # Val loss để theo dõi
    model.eval()
    with torch.no_grad():
        val_logits = model(X_val_tensor)
        val_loss = criterion(val_logits, y_val_tensor).item()
        val_losses.append(val_loss)

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d}/{MAX_EPOCHS} — train_loss: {loss.item():.4f}, val_loss: {val_loss:.4f}")


In [ ]:
# Cell 5 — Đánh giá trên test set & báo cáo chỉ số
model.eval()
with torch.no_grad():
    test_logits = model(X_test_tensor)
    test_probs = torch.sigmoid(test_logits).numpy().flatten()
    test_preds = (test_probs >= 0.5).astype(int)

# Tính chỉ số
acc = accuracy_score(y_test, test_preds)
prec = precision_score(y_test, test_preds)
rec = recall_score(y_test, test_preds)
f1 = f1_score(y_test, test_preds)
roc_auc = roc_auc_score(y_test, test_probs)
cm = confusion_matrix(y_test, test_preds)
report = classification_report(y_test, test_preds, target_names=target_names)

metrics_df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1", "ROC AUC"],
    "Value": [acc, prec, rec, f1, roc_auc]
})

print("Perceptron — Metrics (Test):")
display(metrics_df)
print("\nClassification Report:\n", report)


In [ ]:
# Cell 6 — Vẽ ROC Curve & lưu hình
fpr, tpr, _ = roc_curve(y_test, test_probs)

plt.figure(figsize=(7, 6))
plt.plot(fpr, tpr, label=f"Perceptron (AUC = {roc_auc:.3f})", lw=2)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.6)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve — Perceptron")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


In [ ]:
# Cell 7 — Ma trận nhầm lẫn & lưu hình
fig, ax = plt.subplots(figsize=(6, 5))
cax = ax.matshow(cm, cmap='Blues')
plt.title("Confusion Matrix — Perceptron")
plt.xlabel("Predicted")
plt.ylabel("Actual")
for (i, j), val in np.ndenumerate(cm):
    ax.text(j, i, val, ha='center', va='center')
plt.colorbar(cax)
plt.tight_layout()
plt.show()


In [ ]:
# Cell 8 — Nâng cao: So sánh với Logistic Regression (baseline)
log_reg = LogisticRegression(
    penalty="l2",
    C=1.0,
    solver="lbfgs",
    max_iter=1000,
    random_state=SEED
)
log_reg.fit(X_train, y_train)

# Val để tham khảo
val_probs_lr = log_reg.predict_proba(X_val)[:, 1]
val_preds_lr = (val_probs_lr >= 0.5).astype(int)
val_f1_lr = f1_score(y_val, val_preds_lr)

# Test
test_probs_lr = log_reg.predict_proba(X_test)[:, 1]
test_preds_lr = (test_probs_lr >= 0.5).astype(int)
acc_lr = accuracy_score(y_test, test_preds_lr)
prec_lr = precision_score(y_test, test_preds_lr)
rec_lr = recall_score(y_test, test_preds_lr)
f1_lr = f1_score(y_test, test_preds_lr)
roc_auc_lr = roc_auc_score(y_test, test_probs_lr)
cm_lr = confusion_matrix(y_test, test_preds_lr)

metrics_lr_df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1", "ROC AUC"],
    "Perceptron": [acc, prec, rec, f1, roc_auc],
    "LogisticRegression": [acc_lr, prec_lr, rec_lr, f1_lr, roc_auc_lr]
})

print("So sánh Perceptron vs Logistic Regression — Metrics (Test):")
display(metrics_lr_df)

# ROC so sánh
fpr_lr, tpr_lr, _ = roc_curve(y_test, test_probs_lr)
plt.figure(figsize=(7, 6))
plt.plot(fpr, tpr, label=f"Perceptron (AUC = {roc_auc:.3f})", lw=2)
plt.plot(fpr_lr, tpr_lr, label=f"LogReg (AUC = {roc_auc_lr:.3f})", lw=2)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.6)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve — Perceptron vs Logistic Regression")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


In [ ]:
# Cell 9 — Nâng cao: Phân tích đặc trưng quan trọng (coef)
# Với Logistic Regression: hệ số là tuyến tính, dễ diễn giải
coefs = log_reg.coef_.flatten()
coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs,
    "abs_coef": np.abs(coefs)
}).sort_values("abs_coef", ascending=False)

print("Top đặc trưng (theo |coef|) — Logistic Regression:")
display(coef_df.head(10))

# Với Perceptron: lấy trọng số từ nn.Linear
w = model.linear.weight.detach().numpy().flatten()
b = model.linear.bias.detach().numpy().item()
w_df = pd.DataFrame({
    "feature": feature_names,
    "weight": w,
    "abs_weight": np.abs(w)
}).sort_values("abs_weight", ascending=False)

print("Top đặc trưng (theo |weight|) — Perceptron:")
display(w_df.head(10))


In [ ]:
# Cell 10 — Lưu bảng chỉ số & báo cáo (tuỳ chọn)
# Lưu bảng chỉ số Perceptron
metrics_df.to_csv("perceptron_metrics.csv", index=False)

# Lưu classification report
with open("perceptron_classification_report.txt", "w") as f:
    f.write(report)

# Lưu bảng so sánh
metrics_lr_df.to_csv("comparison_metrics.csv", index=False)

# Lưu top đặc trưng
coef_df.to_csv("logreg_feature_importance.csv", index=False)
w_df.to_csv("perceptron_feature_importance.csv", index=False)

print("Đã lưu các bảng chỉ số và báo cáo.")


Khi nhìn vào ROC-AUC và F1,  sẽ thấy bài toán này “mở lòng” với các mô hình tuyến tính: cấu trúc đặc trưng đã được chuẩn hoá và mối quan hệ giữa các biến với nhãn khá gần tuyến tính, nên Logistic Regression thường nhỉnh hơn hoặc tương đương Perceptron thuần SGD. Điều này hợp lý: Logistic Regression tối ưu log-loss với thuật toán cấp tiến (LBFGS) ổn định, trong khi Perceptron dùng SGD trên BCEWithLogitsLoss có thể hội tụ chậm hơn với lr nhỏ, hoặc dao động với lr lớn.

Hiệu năng tổng quan: ROC-AUC thường cao (gần 0.99) trên bộ dữ liệu này. Nếu F1 và Recall cao, mô hình nhạy với ung thư ác tính — điều quan trọng về lâm sàng. Tuy nhiên Precision cũng cần giữ vững để hạn chế false positives và giảm áp lực xét nghiệm bổ sung.

Ma trận nhầm lẫn: Kiểm tra số lượng FN (ác tính bị dự đoán là lành) — đây là lỗi “đau” nhất. Nếu FN thấp, mô hình đang ưu tiên an toàn cho bệnh nhân, chấp nhận thêm vài FP.

So sánh mô hình: Logistic Regression thường đạt ROC-AUC và F1 nhỉnh hơn một chút. Nếu Perceptron kém hơn, thử lr = 1e-1 (vẫn trong ràng buộc) hoặc thêm weight decay rất nhỏ để ổn định; song yêu cầu bài toán không đặt nặng regularization nên giữ nguyên là hợp lệ.

Đặc trưng quan trọng: Các đặc trưng liên quan đến “mean radius”, “mean texture”, “worst perimeter”, “mean concavity”, “worst concave points” thường xuất hiện với |coef| hoặc |weight| lớn. Dấu hệ số giúp ta hiểu chiều ảnh hưởng: hệ số dương đẩy xác suất về ác tính, hệ số âm về lành tính.

Giới hạn: Mô hình tuyến tính không nắm bắt tương tác phi tuyến mạnh; tuy nhiên với dữ liệu này, tuyến tính đã đủ tốt. Với dữ liệu khác, có thể xem xét thêm polynomial features hoặc kernel methods.

Kết luận thực tế:  Logistic Regression là ứng viên đầu tiên. Perceptron chuẩn tắc vẫn đạt tốt, và là bước khởi đầu gọn nhẹ cho việc chuyển lên các mạng sâu hơn nếu cần.